In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install librosa pydub noisereduce soundfile

In [ ]:
import os
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import noisereduce as nr
import csv
import soundfile as sf

In [ ]:
is_Mohan = True

In [ ]:
if is_Mohan:
  audio_dir = '/content/drive/MyDrive/MHA_Proj_Data/Project_Dataset_Release/AUDIO'
  metadata_path = '/content/drive/MyDrive/MHA_Proj_Data/Project_Dataset_Release/metadata.csv'
  test_base_path = '/content/drive/MyDrive/MHA_Proj_Data/Project_Dataset_Release/LISTS/'
else:
  audio_dir = '/content/drive/MyDrive/Project_Dataset_Release/AUDIO'
  metadata_path = '/content/drive/MyDrive/Project_Dataset_Release/metadata.csv'

In [ ]:
id_to_gender = {}
id_to_status = {}

with open(metadata_path, newline='\n') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        row_split = row[0].split(' ')
        if row_split[0] == 'SUB_ID':
            continue
        id_to_gender[row_split[0]] = row_split[2]
        id_to_status[row_split[0]] = row_split[1]

In [ ]:
id_to_gender['FONIOYTJ']

'm'

In [ ]:
train_files = ['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
valid_files = ['val_0.csv', 'val_1.csv', 'val_2.csv', 'val_3.csv', 'val_4.csv']

In [ ]:
def file_to_IDs(file):
    file = test_base_path + file
    res = []

    with open(file, newline='\n') as file:
      reader = csv.reader(file)
      for row in reader:
        res.append(row[0])

    return res

In [ ]:
"""
Step 1: You are given an ID:
Step 2: Get three audio samples for the ID from AUDIO's breathing, cough, and speech.
Step 3: Extract features from each samples.
Step 3.1: If it's a positive sample, do augmentation and extract features. Why? To balance +ve and -ve samples.
Step 4: Return a list of elements. Each element is further a list with two elements (feature vector and status value)
"""
ID = 'FONIOYTJ'
Datadir = audio_dir

In [ ]:
"""
Data Augmentation ideas:

1. Time strech. (Twice as fast and half the original speed)
2. Pitch shift. (Shift by a tritone for now)
"""

def augmentSignal(y, sr):
    try:
      y_fast = librosa.effects.time_stretch(y, rate=2.0)
      y_slow = librosa.effects.time_stretch(y, rate=0.5)
      y_tritone = librosa.effects.pitch_shift(y, sr=sr, n_steps=-6)
    except:
      return None
    return [y, y_fast, y_slow, y_tritone]

In [ ]:
def intervalsToSignals(intervals, y):
    n, _ = intervals.shape
    res = []
    for i in range(0, n):
        res.append(y[intervals[i][0]:intervals[i][1]])
    return res

In [ ]:
def displaySignal(y):
    plt.figure()
    librosa.display.waveshow(y=y)
    plt.xlabel("Time (xsecs) ")
    plt.ylabel("Amplitude")
    plt.show()

In [ ]:
def ID_to_featureVector(ID, train=False):
    breathing_y, breathing_s = librosa.load(Datadir + '/breathing/' + ID + '.flac')
    cough_y, cough_s = librosa.load(Datadir + '/cough/' + ID + '.flac')
    speech_y, speech_s = librosa.load(Datadir + '/speech/' + ID + '.flac')

    # Denoise the signal
    breathing_y = nr.reduce_noise(y=breathing_y, sr=breathing_s)
    cough_y = nr.reduce_noise(y=cough_y, sr=cough_s)
    speech_y = nr.reduce_noise(y=speech_y, sr=speech_s)

    res = []
    is_f = 1 if (id_to_gender[ID] == 'm') else 0

    ## Augment only if we are training, not during testing!
    if train:
      # If it belongs to a positive patient, augment the signal and split it into non-silent intervals.
      # [[non-silent signal arrays], sr, male or female information, covid positive or negative information]
      if (id_to_status[ID] == 'p'):
          for (y, sr) in [(breathing_y, breathing_s), (cough_y, cough_s), (speech_y, speech_s)]:
              aug_y = augmentSignal(y, sr)
              if aug_y == None:
                continue
              for y_sig in aug_y:
                  intervals = librosa.effects.split(y_sig)
                  sSplit = intervalsToSignals(intervals, y_sig)
                  for signal in sSplit:
                      res.append([signal, sr, is_f])
      else:
          for (y, sr) in [(breathing_y, breathing_s), (cough_y, cough_s), (speech_y, speech_s)]:
              intervals = librosa.effects.split(y)
              sSplit = intervalsToSignals(intervals, y)
              for signal in sSplit:
                  res.append([signal, sr, is_f])
    else:
        for (y, sr) in [(breathing_y, breathing_s), (cough_y, cough_s), (speech_y, speech_s)]:
          aug_y = augmentSignal(y, sr)
          if aug_y == None:
            continue
          for y_sig in aug_y:
            intervals = librosa.effects.split(y_sig)
            sSplit = intervalsToSignals(intervals, y_sig)
            for signal in sSplit:
                res.append([signal, sr, is_f])

    return res

In [ ]:
"""
Obs1: Takes a lot of time to load an ID's three sounds.
Obs2: Have everthing at hand, so they maybe used again in the future.
"""

"\nObs1: Takes a lot of time to load an ID's three sounds.\nObs2: Have everthing at hand, so they maybe used again in the future.\n"

In [ ]:
"""
Extract Features from Processed Signals
"""
def extract_features(signal, sr):
    features = {}
    # Time-domain features
    features['zcr'] = np.mean(librosa.feature.zero_crossing_rate(signal))
    features['rms'] = np.mean(librosa.feature.rms(y=signal))

    # Frequency-domain features
    features['spectral_centroid'] = np.mean(librosa.feature.spectral_centroid(y=signal, sr=sr))
    features['spectral_bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=signal, sr=sr))

    # Mel-frequency cepstral coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
    for i, mfcc in enumerate(mfccs):
        features[f'mfcc_{i+1}'] = np.mean(mfcc)

    return features

In [ ]:
"""
Apply Feature Extraction to All Processed Signals
Use the output of ID_to_featureVector to extract features for every audio segment.
"""
def create_feature_dataset(processed_signals):
    dataset = []
    res = []
    for signal, sr, is_f in processed_signals:
        # Extract features for the current signal
        features = extract_features(signal, sr)
        features['is_female'] = is_f
        #features['is_positive'] = is_p
        dataset.append(features)

    return pd.DataFrame(dataset)

In [ ]:
train_IDs = file_to_IDs('train_0.csv')
val_IDs = file_to_IDs('val_0.csv')

print(train_IDs)

['NPHYFCYK', 'HTCFAFDO', 'XLIOSZXS', 'FONIOYTJ', 'DZEREGIM', 'YQUPZEUJ', 'OFOILIER', 'BLLALITL', 'YLAXTREN', 'HTVKVBPE', 'GYEZORLW', 'SRTPYIDB', 'MSTBAUCK', 'QMJWRIEU', 'BPXVMOVA', 'FYYDNDDH', 'RHDNXKED', 'VBHKKVHJ', 'WJGZILBZ', 'MBXASRGM', 'OZGVGHCG', 'PMEGFZMU', 'BKCTGCKU', 'OLOJOLQJ', 'QEJJIHWU', 'VBXMIXBC', 'DAOJYZLP', 'VBGUQIFD', 'OOFLPSLD', 'OBZRGYUB', 'MGBAKYDP', 'LNFUNOIQ', 'QWZGKNXN', 'OFVPVUNZ', 'UHUKGHOL', 'SSGGLNQE', 'PRVQTLFF', 'BOPYQXJZ', 'EHEKWVXF', 'TMWZXJNT', 'MHSTXVSO', 'ERYUZYCO', 'HXWFDUNS', 'TZCBTMVG', 'NAKJKWOE', 'BYLIIJIF', 'RKEWQQUP', 'SHMLQOAR', 'GDLUFIGU', 'MJOTTQOC', 'TBYLKZRC', 'IMXXDFSZ', 'CZYRAAVU', 'HZOOWHAU', 'QUEKGCAE', 'QXAOCRAB', 'AHXDHEST', 'TUSNQCSS', 'RHHOTDPX', 'MCNAFCEF', 'YFJNNLJB', 'KKCMMQPS', 'NWKKSYMO', 'BTYFGFGB', 'EADBNFIG', 'FHIRVICU', 'ONOTPBTG', 'MFFYUDJV', 'BJJYNQDQ', 'AHTQDTIL', 'ACLAPDXF', 'LKVIJLIA', 'QATIJZYL', 'MDNRVGIE', 'GHBRVBQV', 'EVKWRRBM', 'HNDBYDNB', 'ZFXYSDHQ', 'RFGZBAAP', 'TKPLQJNH', 'PDIDQEEN', 'ZNBDPXVG', 'XRACSBQI', 'VI

In [ ]:
features_df = []
positives_df = []

doneIDs = []
fdf = {}

In [ ]:
for ID in train_IDs:
  print(ID)

  if ID in doneIDs:
    features_df.append(fdf[ID])
    positives_df.append(0 if id_to_status[ID] == 'n' else 1)

  fV = ID_to_featureVector(ID, train=True)
  fdf[ID] = create_feature_dataset(fV)
  features_df.append(fdf[ID])
  positives_df.append(0 if id_to_status[ID] == 'n' else 1)



NPHYFCYK
HTCFAFDO
XLIOSZXS


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=512
  warnings.warn(


FONIOYTJ
DZEREGIM


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1536
  warnings.warn(


YQUPZEUJ
OFOILIER
BLLALITL
YLAXTREN
HTVKVBPE
GYEZORLW
SRTPYIDB
MSTBAUCK
QMJWRIEU
BPXVMOVA
FYYDNDDH
RHDNXKED
VBHKKVHJ
WJGZILBZ


...

XEQWVUAB


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1165
  warnings.warn(


AWBEGKDR
SGISFSZA
GSCDSZVF
RCNCGHRA


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1677
  warnings.warn(


VDYIGEGY
ILQLERFN
JLISAUWH
WXWPGFPH
RCROGMKQ
YAFNLTYD
IFPRPNRB
VVXAOBIQ
RSNVHYQJ
FGYDTCSG
RTHBTMSA
FXTZOQME
VBIAHODB
KXYGVTKN
EJVCCKSN
IMVMQMPG
PADSFUKF


In [ ]:
ID_to_featureVector('YAFNLTYD', train=True)

/usr/local/lib/python3.10/dist-packages/noisereduce/spectralgate/nonstationary.py:70: RuntimeWarning: invalid value encountered in divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth


[[array([ 8.50879442e-05, -2.43988543e-05, -1.03736274e-04, ...,
         -4.37469062e-05, -8.44547867e-06,  3.85009553e-05], dtype=float32),
  22050,
  1],
 [array([1.1119612e-08, 5.6283880e-09, 2.1412763e-09, ..., 8.0966748e-07,
         8.4774325e-07, 4.4400770e-07], dtype=float32),
  22050,
  1],
 [array([-2.5313232e-06, -3.0254421e-06, -3.7649718e-06, ...,
          1.6062770e-14,  2.3538700e-14,  3.0370458e-14], dtype=float32),
  22050,
  1],
 [array([ 3.8236496e-07,  3.8537010e-07,  5.1120418e-07, ...,
         -9.8426472e-07, -9.5973439e-07, -9.3361092e-07], dtype=float32),
  22050,
  1],
 [array([1.9477568e-08, 1.4709797e-08, 1.8436301e-08, ..., 1.0738692e-07,
         7.7865835e-08, 3.3207439e-08], dtype=float32),
  22050,
  1],
 [array([ 4.2106092e-04,  3.6628955e-04,  2.6701743e-04, ...,
         -7.4518164e-07, -1.0256946e-06, -1.0633938e-06], dtype=float32),
  22050,
  1],
 [array([ 3.9121002e-18,  2.9570079e-14,  1.0819467e-13, ...,
          7.9971952e-14, -2.5236760e-1